In [2]:
import json
from transformers import pipeline

# Charger les données clusterisées
def load_clustered_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Chemin vers le fichier clusterisé
clustered_file_path = 'clustered_articles2.json'
clustered_data = load_clustered_data(clustered_file_path)

# Regrouper les articles par cluster
clusters = {}
for article in clustered_data:
    cluster_id = article['cluster']
    if cluster_id not in clusters:
        clusters[cluster_id] = []
    clusters[cluster_id].append(article['content'])

# Charger un modèle de génération de résumé (BART)
try:
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
except Exception as e:
    print(f"Erreur lors du chargement du modèle: {e}")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)  # Forcer l'utilisation du CPU

# Générer un résumé pour chaque cluster
cluster_summaries = {}
for cluster_id, contents in clusters.items():
    # Concaténer le contenu de tous les articles du cluster
    combined_content = " ".join(contents)
    
    # Générer un résumé (limité à 1024 tokens pour BART)
    max_input_length = 1024
    if len(combined_content) > max_input_length:
        combined_content = combined_content[:max_input_length]
    
    try:
        summary = summarizer(combined_content, max_length=130, min_length=30, do_sample=False)
        cluster_summaries[cluster_id] = summary[0]['summary_text']
    except Exception as e:
        print(f"Erreur lors de la génération du résumé pour le cluster {cluster_id}: {e}")
        cluster_summaries[cluster_id] = "Résumé non disponible"

# Sauvegarder les résumés dans un fichier JSON
output_file_path = 'cluster_summaries_bart.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(cluster_summaries, file, ensure_ascii=False, indent=4)

print(f"Les résumés ont été enregistrés dans '{output_file_path}'.")

/home/oumayma/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
Device set to use cpu


Les résumés ont été enregistrés dans 'cluster_summaries_bart.json'.
